In [38]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [39]:
import langchain

In [44]:
os.environ["GOOGLE_API_KEY"]=os.getenv("GOOGLE_API_KEY")

Example 1: Simple LLM Call With Streaming

In [6]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage,SystemMessage

In [51]:
model=init_chat_model("gemini-2.5-flash",model_provider="google_genai")
model

ChatGoogleGenerativeAI(model='models/gemini-2.5-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x1583d9b80>, default_metadata=(), model_kwargs={})

In [52]:
## Create Message
messages=[
    SystemMessage("You are a helpful AI Assistant"),
    HumanMessage("What are the top 2 benefits of using Langchain?")

]

In [53]:
## invoke the model
response=model.invoke(messages)
print(response.content)

LangChain is a powerful framework for developing applications powered by large language models (LLMs). Its top two benefits are:

1.  **Enabling Context-Aware and Interactive Applications:**
    LangChain excels at allowing LLMs to go beyond just generating text based on their training data. It provides tools to:
    *   **Connect LLMs to external data sources:** Retrieve up-to-date information from databases, documents, or the web (e.g., RAG - Retrieval Augmented Generation).
    *   **Allow LLMs to interact with external tools/APIs:** Empower LLMs to perform actions like sending emails, running code, searching specific websites, or interacting with other software (e.g., through Agents).
    This capability transforms LLMs from static text generators into dynamic, knowledgeable, and actionable agents.

2.  **Streamlining Development and Reducing Boilerplate:**
    LangChain provides a set of modular components and a structured way to combine them, significantly accelerating the develo

In [54]:
resp=model.invoke([HumanMessage("How to make a career transition from QA to PM")])
print(resp.content)

Transitioning from a Quality Assurance (QA) role to a Product Manager (PM) position is a common and often successful career path. Your QA experience provides a unique and valuable perspective that many PMs lack, particularly in understanding product quality, user pain points, and technical feasibility.

Here's a comprehensive guide on how to make that transition:

---

## Why QA to PM is a Natural Fit

Before diving into the "how," let's acknowledge your existing strengths:

1.  **Deep Product Knowledge:** You know the product inside out, its features, limitations, and common user flows.
2.  **User Empathy:** You constantly think about the user's experience and how bugs impact them.
3.  **Attention to Detail:** Critical for identifying issues and ensuring high-quality releases.
4.  **Analytical Thinking:** You're skilled at breaking down complex systems and identifying root causes.
5.  **Risk Assessment:** You understand potential failure points and their impact.
6.  **Cross-Functional

In [ ]:
## Streaming Example
for chunk in model.stream(messages):
    print(chunk.content,end="",flush=True)

The top two benefits of using LangChain are:

1.  **Streamlined Development & Modularity:** LangChain provides a structured, modular framework that significantly simplifies and accelerates the development of complex LLM applications. It offers pre-built components (like chains, agents, prompt templates, memory, document loaders) that can be easily combined, customized, and reused. This reduces boilerplate code, promotes best practices, and makes it much faster to prototype and deploy sophisticated LLM-powered solutions.

2.  **Enhanced LLM Capabilities & External Integration:** LangChain's core power lies in its ability to connect LLMs to external data sources and tools. This allows LLMs to:
    *   **Access up-to-date and specific information** (e.g., via Retrieval Augmented Generation - RAG) that wasn't in their training data.
    *   **Perform actions in the real world** (e.g., using agents to browse the web, execute code, call APIs, or interact with databases).
    *   This overcom

## Dynamic Prompt Templates

In [56]:
from langchain_core.prompts import ChatPromptTemplate

## create a translation app
translation_template=ChatPromptTemplate.from_messages([
    ("system","You are a Professional translator. Translate the following {text} to {source_language} to {target_language}.Maintain the tone and style"),
    ("user","{text}")

])

# use this template
prompt=translation_template.invoke({
    "source_language":"English",
    "target_language":"Vietnamese",
    "text":"Hello"
})


In [57]:
translated_response=model.invoke(prompt)
print(translated_response.content)

Chào bạn!
(This is a common, friendly way to say "Hello" back. If you just want the translation of "Hello" in Vietnamese, it's "Xin chào.")

Here are a few options for "Hello" in Vietnamese, depending on context:

*   **Xin chào:** (Most common, polite, and neutral way to say "Hello" or "Hi" to anyone.)
*   **Chào bạn:** (Friendly "Hello" to a peer or friend.)
*   **Chào anh/chị/em/ông/bà:** (More specific, polite "Hello" depending on the age and gender of the person you're addressing.)


## Building Your First Chain

In [58]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough,RunnableLambda
def create_story_chain():
    # Template for story generation
    story_prompt=ChatPromptTemplate.from_messages(
        [
            ("system","You are creative storyteller. Write a short and engaging story based on a given theme character and setting."),
            ("user","Theme:{theme}\nMain character:{character}\nSetting:{setting}")
        ]
    )

    ## Template for story analysis
    analysis_prompt=ChatPromptTemplate.from_messages([
        ("system","You are a literary critic. Analyze the following story and provide insights"),
        ("user","{story}")
    ])
    story_chain=(
        story_prompt| model| StrOutputParser()
    )
    # create a function to pass the story to analysis
    def analyze_story(story_text):
        return {"story":story_text}
    
    analysis_chain=(
        story_chain
        |RunnableLambda(analyze_story)
        |analysis_prompt
        |model
        |StrOutputParser()
    )
    return analysis_chain

In [59]:
chain=create_story_chain()
chain

ChatPromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are creative storyteller. Write a short and engaging story based on a given theme character and setting.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, template='Theme:{theme}\nMain character:{character}\nSetting:{setting}'), additional_kwargs={})])
| ChatGoogleGenerativeAI(model='models/gemini-2.5-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x1583d9b80>, default_metadata=(), model_kwargs={})
| StrOutputParser()
| RunnableLambda(analyze_story)
| ChatPromptTemplate(input_variables=['story'], input_types={}, partial_variab

In [60]:
result= chain.invoke({
        "theme":"artificial intelligence",
        "character":"a curious robot",
        "setting":"a futurastic city"
    })
print("Story and Analysis:")
print(result)

Story and Analysis:
This poignant short story, "Bolt," offers a compelling exploration of consciousness, purpose, and the inherent value of the natural world in a meticulously engineered future. It masterfully uses classic science fiction tropes to delve into profound philosophical questions, all within a concise and evocative narrative.

Here's a deeper analysis:

**1. Character Analysis: Bolt's Emergence of Consciousness**

*   **The Archetype:** Bolt begins as the quintessential highly specialized robot: "City Surveyor Bot, Unit 734, sleek and chrome," designed for efficiency and data processing. He represents the pinnacle of artificial intelligence in a controlled environment.
*   **The Spark of Curiosity:** The key to Bolt's transformation is his "curiosity." This isn't just a programmed function; it's presented as an intuitive, almost human-like inclination. His optical sensors "lingered on more than just girders and conduits," suggesting an aesthetic appreciation or an innate de